In [1]:
import json
import os
import time
import re
import pandas as pd
import string


In [2]:
a = { "in_reply_to_status_id": None}
if a["in_reply_to_status_id"]!=None:
    print (1)

In [3]:

def twitter_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    selected_feature["id"] = json_file["id"]
    selected_feature["created_at"] = json_file["created_at"]
    if "referenced_tweets" in json_file.keys():   
        selected_feature["referenced_tweets"] = json_file["referenced_tweets"][0]["id"]
    selected_feature['retweet_count'] = json_file['public_metrics']['retweet_count']
    
    # selected_feature["hashtag"] = re.search(r'(\w+)+', json_file["text"])
    return selected_feature

def test_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    selected_feature["created_at"] = json_file["created_at"]
    if json_file["in_reply_to_status_id_str"]!= None:   
        selected_feature["referenced_tweets"] = json_file["in_reply_to_status_id_str"]
    selected_feature["id"] = json_file["id_str"]
    selected_feature['retweet_count'] = json_file['retweet_count']

    return selected_feature

def convert_label(label):
    if label == "rumour":
        return 1
    elif label == "nonrumour":
        return 0 
def merge_dictionary_list(dict_list):
      return {
    k: [d.get(k) for d in dict_list if k in d]
    for k in set().union(*dict_list)
  } 
def remove_unwanted_keys(dict_list, keys):
    for item in dict_list:
        item.pop(keys, None)
    return dict_list

In [4]:
def getgraph(events):
    graph = {}
    for e in events:   
        graph[e["id"]]=[]

    for e in events:
        if "referenced_tweets" in e.keys():
            #str
            k = e["referenced_tweets"]
            v = e["id"]
            if k in graph.keys(): 
                graph[k].append(e)
            else:
                graph[k] = [e]
    return(graph)
                
            




# time_sequence = []

# ## sort dfs as the reply order
# ##everynode type is (id,text,time,)
# def dfs(visited, graph, node):
    
#     if node["id"] not in visited:
#         time_sequence.append(node)
# #         print (node)
#         visited.add(node["id"])
#         for neighbour in graph[node["id"]]:
            
#             dfs(visited, graph, neighbour)

# eventg= getgraph(testevent)
# # # Driver Code
# dfs(visited, eventg, sourcetweet)
# print (time_sequence)




## sort dfs as the reply order
##everynode type is (id,text,time,)
def dfs(visited, graph, node,time_sequence):
    
    if node["id"] not in visited:
        time_sequence.append(node)
#         print (node)
        visited.add(node["id"])
        for neighbour in graph[node["id"]]:
            
            dfs(visited, graph, neighbour,time_sequence)
    return time_sequence



In [5]:
def load_data(file_name):
    if (file_name == 'train' or file_name == 'dev'):
        total_json = []
        finished_dict = []
        with open("/Users/yuweng/Desktop/task1/" + file_name +".data.txt", "r") as f:
            file_name_event = f.readlines()      
        with open("/Users/yuweng/Desktop/task1/" + file_name +".label.txt", "r") as f:
            file_name_label = f.readlines()
        for event_str, event_label in zip(file_name_event, file_name_label):
            event_list = event_str.rstrip().split(",")
            event_label = event_label.rstrip().split(",")
            # print(type(event_label))
            event_json = []
            path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + event_list[0] + ".json"
            # print(path)
            # print(os.path.exists(path))
            if os.path.exists(path):
                for id in event_list:
                    path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + id + ".json"
                    # print(path)
                    # print(os.path.exists(path))
                    if os.path.exists(path):
                        related_tweet = json.load(open(path, "r"))
                        # print(related_tweet)
                        event_json.append(twitter_feature_selection(related_tweet))
                
                event_json = sorted(event_json, key=lambda x: time.mktime(time.strptime(x["created_at"], '%Y-%m-%dT%H:%M:%S.%fZ')))
                sourcetweet = event_json[0]
                eventgraph = getgraph(event_json)
                visited = set()
                sortresult = []
                event_json = dfs(visited, eventgraph, sourcetweet,sortresult)
                
                event_json.append({"label":convert_label(''.join(event_label))})
                # print(event_json)
                merged_dict = merge_dictionary_list(event_json)
                # print(merged_dict)
                total_json.append(merged_dict)
                # print(total_json)
                finished_dict = remove_unwanted_keys(total_json, "created_at")
    elif (file_name == 'test'):
        total_json = []
        finished_dict = []
        with open("/Users/yuweng/Desktop/task1/" + file_name +".data.txt", "r") as f:
            file_name_event = f.readlines()      
        for event_str in file_name_event:
            event_list = event_str.rstrip().split(",")
            event_json = []
            path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + event_list[0] + ".json"
            # print(path)
            # print(os.path.exists(path))
            if os.path.exists(path):
                for id in event_list:
                    path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + id + ".json"
                    # print(path)
                    # print(os.path.exists(path))
                    if os.path.exists(path):
                        related_tweet = json.load(open(path, "r"))
                        # print(related_tweet)
                        event_json.append(test_feature_selection(related_tweet))
                event_json = sorted(event_json, key=lambda x: time.mktime(time.strptime(x["created_at"], '%a %b %d %H:%M:%S %z %Y')))
                # print(event_json)
                sourcetweet = event_json[0]
                eventgraph = getgraph(event_json)
                visited = set()
                sortresult = []
                event_json = dfs(visited, eventgraph, sourcetweet,sortresult)
                
                merged_dict = merge_dictionary_list(event_json)
                # print(merged_dict)
                total_json.append(merged_dict)
                # print(total_json)
                finished_dict = remove_unwanted_keys(total_json, "created_at")
    return pd.DataFrame(finished_dict)

Load the Required data set

In [6]:
df_train = load_data("train")
df_dev = load_data("dev")
df_test = load_data("test")

In [7]:
def text_preprocessing(tweet):
    r1 = r'https?:/\/\S+'
    ## remove @people
    r2 = r'@[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'
    ##remove tag
    r3 = r'#[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'

    tweet = tweet.encode('utf-8', 'replace').decode('utf-8')
    tweet = re.sub(r1, "", tweet)
    tweet = re.sub(r2, "", tweet)
    tweet = re.sub(r3, "", str(tweet).lower().strip())
    return tweet
def give_emoji_free_text(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F1F2-\U0001F1F4"  
        u"\U0001F1E6-\U0001F1FF" 
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table).strip()


Join the Text together and do the data preprocessing

In [8]:
df_dev['text'][15]

['Google South American Goliath Birdeater. HELLLLL NOOOOOO✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋',
 '@snooki 😩😫😩😫😩 they just keep finding stuff',
 '@snooki not fallin for that again',
 '@snooki nasty!!!!!! 😭',
 '@snooki no reason it should even exist 😱😱😱😱😱😩😩',
 '@snooki ok',
 '@snooki NOOOOO 😷',
 '@snooki WTF A PUPPY SIZED SPIDER',
 'THIS IS THE SPIDER \n\n“@snooki: Google South American Goliath Birdeater. HELLLLL NOOOOOO✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋”',
 "@snooki NO NO NO NO NO NO! That's a horror movie spider!",
 "@snooki lol! I'm the same way with rats....",
 '@snooki I thought it was going to be a bird😭😭😭😭',
 '@snooki BYEEEEE 👋👋👋👋👋👋',
 '“@snooki: Google South American Goliath Birdeater. HELLLLL NOOOOOO✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋” I JUST CRINGED!!!',
 '"@snooki: Google South American Goliath Birdeater. HELLLLL NOOOOOO✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋✋" why would you do this to me?',
 "@snooki hence why I'm arachnophobic!!!!",
 '@snoo

In [9]:
df_train['text'] = df_train.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
df_train['text'] = df_train['text'].apply(lambda x:x.lower().replace('\n',''))
df_train['text'] = df_train['text'].apply(lambda x:give_emoji_free_text(x))
df_train['label'] = df_train.apply(lambda x:x["label"][0],axis=1)
df_train['label'] = df_train['label'].astype('int')
df_train['retweet_count'] = df_train.apply(lambda x:sum(x["retweet_count"]),axis=1)

df_dev['text'] = df_dev.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
df_dev['text'] = df_dev['text'].apply(lambda x:x.lower().replace('\n',''))
df_dev['text'] = df_dev['text'].apply(lambda x:give_emoji_free_text(x))
df_dev['label'] = df_dev.apply(lambda x:x["label"][0],axis=1)
df_dev['label'] = df_dev['label'].astype('int')
df_dev['retweet_count'] = df_dev.apply(lambda x:sum(x["retweet_count"]),axis=1)


df_test['text'] = df_test.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
df_test['text'] = df_test['text'].apply(lambda x:x.lower().replace('\n',''))
df_test['text'] = df_test['text'].apply(lambda x:give_emoji_free_text(x))
df_test['retweet_count'] = df_test.apply(lambda x:sum(x["retweet_count"]),axis=1)



df_train.drop(['id','referenced_tweets'],axis=1,inplace=True)
df_dev.drop(['id','referenced_tweets'],axis=1,inplace=True)
df_test.drop(['id','referenced_tweets'],axis=1,inplace=True)


# df_train['text'] = df_train['text'].apply(lambda x:remove_URL(x))

# df_train['text'] = df_train['text'].apply(lambda x:remove_html(x))
# df_train['text'] = df_train['text'].apply(lambda x:remove_punct(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_URL(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_html(x))
# df_dev['text'] = df_dev['text'].apply(lambda x:remove_punct(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_URL(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_html(x))
# df_test['text'] = df_test['text'].apply(lambda x:remove_punct(x))

df_train.dropna(axis=0, how='any', inplace=True)
df_dev.dropna(axis=0, how='any', inplace=True)
df_test.dropna(axis=0, how='any', inplace=True)

In [10]:
df_dev['text'][15]

'google south american goliath birdeater. helllll noooooo  they just keep finding stuff not fallin for that again nasty!!!!!!  no reason it should even exist  ok nooooo  wtf a puppy sized spiderthis is the spider “ google south american goliath birdeater. helllll noooooo” no no no no no no! that\'s a horror movie spider! lol! i\'m the same way with rats.... i thought it was going to be a bird byeeeee “ google south american goliath birdeater. helllll noooooo” i just cringed!!!" google south american goliath birdeater. helllll noooooo" why would you do this to me? hence why i\'m arachnophobic!!!!  .. “ google south american goliath birdeater. helllll noooooo” _bwn  fuck that shit!! gurllll. i got one for you. google camel spider.  omg ah i hate you...   my curiosity got the best of me hahahaha  fuck u why would u make me“ google south american goliath birdeater. helllll noooooo”“ google south american goliath birdeater. helllll noooooo” omg i noped that so hard!  i threw my phone across

In [11]:
df_train.isnull().any()

retweet_count    False
text             False
label            False
dtype: bool

In [12]:
df_train

,retweet_count,text,label
0,0,4. can eating garlic help prevent infection wi...,0
1,185,french police chief killed himself after atta...,1
2,1,coronavirus disease (covid-19) advice for the ...,0
3,123,ottawa police confirm that there were multiple...,0
4,1,if the primary focus of a government isn't to ...,0
...,...,...,...
1558,0,4. it cannot be transmitted through goods manu...,0
1559,83,desperate ted cruz claims planned parenthood s...,1
1560,109,"""thoughts and prayers are not enough."" pres. o...",1
1561,177,police have surrounded this building where the...,0


In [13]:
df_train.to_csv("train_dfs.csv",index= False)
df_dev.to_csv('dev_dfs.csv',index= False)
df_test.to_csv('test_dfs.csv',index= False)

In [14]:
df_train = pd.read_csv('/Users/yuweng/Desktop/task1/train_dfs.csv')
df_dev = pd.read_csv('/Users/yuweng/Desktop/task1/dev_dfs.csv')
df_test = pd.read_csv('/Users/yuweng/Desktop/task1/test_dfs.csv')

In [15]:
df_train.isnull().any()

retweet_count    False
text             False
label             True
dtype: bool

In [16]:
df_train.dropna(axis=0, how='any', inplace=True)
df_dev.dropna(axis=0, how='any', inplace=True)
df_test.dropna(axis=0, how='any', inplace=True)

In [17]:
df_train.isnull().any()

retweet_count    False
text             False
label            False
dtype: bool

In [18]:
df_train.to_csv("train_dfs.csv",index= False)
df_dev.to_csv('dev_dfs.csv',index= False)
df_test.to_csv('test_dfs.csv',index= False)